In [1]:
# Ячейка 1: ПРИНУДИТЕЛЬНОЕ ОБНОВЛЕНИЕ БИБЛИОТЕК
!apt-get update -qq
!apt-get install -y -qq djvulibre-bin tesseract-ocr tesseract-ocr-rus tesseract-ocr-deu tesseract-ocr-eng
# Флаг -U обязателен, чтобы получить доступ к gemini-1.5-flash
!pip install -U -q google-generativeai telethon pymupdf tqdm pillow pytesseract
!echo "✅ Библиотеки ОБНОВЛЕНЫ. Теперь модель должна быть видна."

^C
Selecting previously unselected package djvulibre-bin.
(Reading database ... 128639 files and directories currently installed.)
Preparing to unpack .../0-djvulibre-bin_3.5.28-2ubuntu0.22.04.1_amd64.deb ...
Unpacking djvulibre-bin (3.5.28-2ubuntu0.22.04.1) ...
Selecting previously unselected package libexiv2-27:amd64.
Preparing to unpack .../1-libexiv2-27_0.27.5-3ubuntu1_amd64.deb ...
Unpacking libexiv2-27:amd64 (0.27.5-3ubuntu1) ...
Selecting previously unselected package libgraphicsmagick-q16-3.
Preparing to unpack .../2-libgraphicsmagick-q16-3_1.4+really1.3.38-1ubuntu0.1_amd64.deb ...
Unpacking libgraphicsmagick-q16-3 (1.4+really1.3.38-1ubuntu0.1) ...
Selecting previously unselected package libgraphicsmagick++-q16-12.
Preparing to unpack .../3-libgraphicsmagick++-q16-12_1.4+really1.3.38-1ubuntu0.1_amd64.deb ...
Unpacking libgraphicsmagick++-q16-12 (1.4+really1.3.38-1ubuntu0.1) ...
Selecting previously unselected package tesseract-ocr-deu.
Preparing to unpack .../4-tesseract-ocr-de

In [2]:
# Ячейка Диагностики
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

print("🔍 Сканирование доступных моделей...")
try:
    user_secrets = UserSecretsClient()
    GENAI_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=GENAI_KEY)
    
    available_models = []
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f"   ✅ Доступна: {m.name}")
            available_models.append(m.name)
            
    if not available_models:
        print("❌ Нет доступных моделей! Проверьте API Key.")
    else:
        print(f"\nВсего найдено: {len(available_models)}")

except Exception as e:
    print(f"❌ Критическая ошибка: {e}")
    print("💡 СОВЕТ: Нажмите в меню 'Run' -> 'Restart Session', чтобы обновить библиотеки.")

🔍 Сканирование доступных моделей...
   ✅ Доступна: models/gemini-2.5-flash
   ✅ Доступна: models/gemini-2.5-pro
   ✅ Доступна: models/gemini-2.0-flash-exp
   ✅ Доступна: models/gemini-2.0-flash
   ✅ Доступна: models/gemini-2.0-flash-001
   ✅ Доступна: models/gemini-2.0-flash-exp-image-generation
   ✅ Доступна: models/gemini-2.0-flash-lite-001
   ✅ Доступна: models/gemini-2.0-flash-lite
   ✅ Доступна: models/gemini-2.0-flash-lite-preview-02-05
   ✅ Доступна: models/gemini-2.0-flash-lite-preview
   ✅ Доступна: models/gemini-2.0-pro-exp
   ✅ Доступна: models/gemini-2.0-pro-exp-02-05
   ✅ Доступна: models/gemini-exp-1206
   ✅ Доступна: models/gemini-2.5-flash-preview-tts
   ✅ Доступна: models/gemini-2.5-pro-preview-tts
   ✅ Доступна: models/learnlm-2.0-flash-experimental
   ✅ Доступна: models/gemma-3-1b-it
   ✅ Доступна: models/gemma-3-4b-it
   ✅ Доступна: models/gemma-3-12b-it
   ✅ Доступна: models/gemma-3-27b-it
   ✅ Доступна: models/gemma-3n-e4b-it
   ✅ Доступна: models/gemma-3n-e2b-it


In [3]:
# === ЯЧЕЙКА 2: V57.1 (FULL META + JSON REPAIR) ===
import os
import time
import json
import asyncio
import shutil
import re
import fitz  # PyMuPDF
import subprocess
import google.generativeai as genai
from PIL import Image
import pytesseract
from datetime import datetime
from telethon import TelegramClient, events, functions, types
from telethon.sessions import MemorySession
from kaggle_secrets import UserSecretsClient
from tqdm.notebook import tqdm
from google.api_core.exceptions import ResourceExhausted, ServiceUnavailable, InternalServerError

# === SAFETY ===
Image.MAX_IMAGE_PIXELS = None 

# === SETTINGS ===
SOURCE_USERNAME = "sendDoc39"
# Стратегия: пробуем 5, при ошибке 3, при совсем ошибке 1
BATCH_STRATEGY = [5, 3, 1]  

GLOBAL_REQUEST_DELAY = 30.0  

ZOOM_FACTOR = 2.0           
MARKER_EMOJI = "✅"
MAX_EMPTY_GAP = 5
SAFETY_LIMIT = 500

DJVU_TARGET_SIZE = 2560     
CROP_PADDING = 15           
SNAP_THRESHOLD = 30         

# --- PRODUCTION SETTINGS ---
DEBUG_PAGE_LIMIT = 11
# ---------------------------

user_secrets = UserSecretsClient()
TEMP_DIR = "./temp_processing"
CROPS_DIR = "./temp_crops"
OCR_JSON_DIR = "./temp_ocr_json"

LOG_BUFFER = []

def log(text):
    tqdm.write(text)
    LOG_BUFFER.append(text)

async def send_report(client, dest_entity):
    report_text = "\n".join(LOG_BUFFER)
    if len(report_text) > 4000: report_text = report_text[-3900:]
    try:
        if dest_entity: await client.send_message(dest_entity, f"📝 **Report:**\n\n...{report_text}")
    except: pass

# === JSON REPAIR KIT ===
def repair_json_content(text):
    """
    Лечит ошибку 'Invalid \escape', экранируя одиночные слеши,
    но сохраняя валидные JSON-последовательности (\n, \t, \", \\).
    """
    # Паттерн находит backslash, за которым НЕ следует валидный escape-символ
    # Valid JSON escapes: ", \, /, b, f, n, r, t, u
    pattern = r'\\(?![/u"bfnrt\\])'
    return re.sub(pattern, r'\\\\', text)

# === MODEL MANAGER ===
class ModelRotator:
    def __init__(self, api_keys):
        if isinstance(api_keys, str): self.api_keys = [api_keys]
        else: self.api_keys = api_keys
        self.current_key_idx = 0
        self._configure_current_key()
        self.last_request_ts = 0
        
        # Priority Models
        self.models_list = [
            "models/gemini-2.5-flash",      # Priority 1
            "models/gemini-2.0-flash-exp",  # Priority 2
            "models/gemini-1.5-flash",      # Priority 3
        ]
        log(f"🧠 Models initialized. Main: {self.models_list[0]}. Delay: {GLOBAL_REQUEST_DELAY}s.")

    def _configure_current_key(self):
        genai.configure(api_key=self.api_keys[self.current_key_idx])

    async def _enforce_global_pace(self):
        now = time.time()
        elapsed = now - self.last_request_ts
        if elapsed < GLOBAL_REQUEST_DELAY:
            wait_time = GLOBAL_REQUEST_DELAY - elapsed + 1
            await asyncio.sleep(wait_time)
        self.last_request_ts = time.time()

    async def generate_content(self, content):
        for i, model_name in enumerate(self.models_list):
            await self._enforce_global_pace()
            
            # Low temp for structural adherence
            model = genai.GenerativeModel(
                model_name=model_name,
                safety_settings=[{"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}],
                generation_config={"response_mime_type": "application/json", "temperature": 0.0}
            )
            
            try:
                log(f"      📡 [{model_name}] Sending...")
                t_start = time.time()
                response = await asyncio.to_thread(model.generate_content, content)
                dur = time.time() - t_start
                
                usage = {"in": 0, "out": 0}
                if response.usage_metadata:
                    usage["in"] = response.usage_metadata.prompt_token_count
                    usage["out"] = response.usage_metadata.candidates_token_count
                
                return response.text, usage, model_name, dur
            
            except (ResourceExhausted, ServiceUnavailable, InternalServerError) as e:
                log(f"      ⚠️ Limit/Error on {model_name}.")
                if i == len(self.models_list) - 1:
                    log("      ❌ ALL MODELS EXHAUSTED. Aborting execution.")
                    raise e 
                log("      ➡️ Switching to backup model...")
                continue
                
            except Exception as e:
                if "429" in str(e):
                    log(f"      ⚠️ 429 Too Many Requests on {model_name}.")
                    if i == len(self.models_list) - 1:
                        log("      ❌ ALL MODELS EXHAUSTED (429). Aborting.")
                        raise e
                    continue
                log(f"      ❌ Unexpected Error ({model_name}): {e}")
                raise e

        raise Exception("Unexpected logic flow in ModelRotator")

# === INIT ===
model_rotator = None
try:
    GENAI_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    model_rotator = ModelRotator(GENAI_KEY)
except Exception as e: log(f"❌ Init Error: {e}")

# === AI LOGIC (UPDATED PROMPT V57.1) ===
def get_gemini_prompt(is_first_batches, previous_context=None):
    base_prompt = """
    TASK: OCR & Layout Analysis (Spread vs Journal Page) with Rich Formatting.
    """
    
    if previous_context:
        base_prompt += f"""
    CONTEXT FROM PREV BATCH: "...{previous_context}..."
    """

    base_prompt += """
    OUTPUT FORMAT (JSON):
    {
      "pages": [
        {
           "book_page_num": "string",         // PRINTED page number.
           "classification_code": "string",   // UDC/BBC codes (e.g. "УДК...").
           "main_text": "string",             // Markdown text with headers/bold/italic.
           "footnotes": "string",             // Isolated footnotes.
           "marginalia": "string",            // Headers/Running titles.
           "media_objects": [
              { "id": "string", "coordinates": [y,x,y,x], "type": "string", "caption_ru": "string" }
           ]
        }
      ],
    """
    
    if is_first_batches:
        base_prompt += """
      "bibliographic_data": {
          "title": "string",
          "authors": ["string"],
          "co_authors": ["string"],
          "publisher": "string",
          "year": "string",
          "isbn": "string",
          "document_type": "string (book, article, official_document, archive_record, other)",
          "historical_period_description": "string (на русском)",
          "historical_year_start": "int or null",
          "historical_year_end": "int or null",
          "summary": "string (на русском)"
      }
    }
    """
    else:
        base_prompt += '  "batch_metadata": { "status": "cont" }\n}'

    # --- RULES V57.1 ---
    base_prompt += """
    CRITICAL RULES:
    
    1. **LAYOUT & ZONES**:
       - Check for **Single Page** vs **Spread** (2 pages).
       - If **Single Page** (Journal Style): Detect horizontal zones (Top Article end / Bottom Article start). Use `***` separator.
       - Do not merge columns across different articles.

    2. **RICH MARKDOWN FORMATTING (HIERARCHY)**:
       - **Headers**: Detect font size/boldness. Use `##` for Main Titles, `###` for Subtitles.
       - **Emphasis**: Use `**bold**` for bold text. Use `*italic*` for italics.
       - **Structure**: Preserve paragraph breaks.

    3. **METADATA**:
       - Extract full bibliographic data if available (first batch).
       - Use **Russian language** for descriptions and summaries.
       - If a value is unknown, use `null` or empty string.

    4. **FOOTNOTES**:
       - CUT footnote text from main body and PASTE into `footnotes`.
       
    5. **JSON SAFETY**:
       - Do not use raw backslashes inside strings unless escaping. Avoid LaTeX macros if possible, use Unicode.
    """
    return base_prompt

async def process_single_batch(image_paths, page_nums, prev_ctx_text):
    if not model_rotator: raise Exception("No models")
    
    is_first_batch = (page_nums[0] == 1)
    
    prompt = get_gemini_prompt(is_first_batch, prev_ctx_text)
    content = [prompt]
    loaded_imgs = []
    
    for p in image_paths:
        img = Image.open(p)
        w, h = img.size
        if w > 3072 or h > 3072:
            img.thumbnail((3072, 3072), Image.Resampling.LANCZOS)
        loaded_imgs.append(img)
        content.append(img)
        
    text_resp, usage, model_name, duration = await model_rotator.generate_content(content)
    
    clean_text = text_resp.replace("```json", "").replace("```", "").strip()
    if clean_text.startswith("json"): clean_text = clean_text[4:].strip()
    match = re.search(r'\{.*\}', clean_text, re.DOTALL)
    if match: clean_text = match.group(0)

    try:
        # 1. Попытка парсинга "как есть"
        data = json.loads(clean_text) 
    except Exception as e1:
        # 2. Попытка лечения JSON (backslashes)
        log(f"      ⚠️ JSON Error 1: {e1}. Repairing...")
        repaired_text = repair_json_content(clean_text)
        try:
            data = json.loads(repaired_text)
            log("      🛠️ Repair Successful!")
        except Exception as e2:
            log(f"      ❌ Repair Failed: {e2}. Raw text len: {len(clean_text)}")
            raise Exception("JSON Parsing Failed completely") 
    
    batch_meta = data.get("bibliographic_data", data.get("batch_metadata", None))
    return data.get("pages", []), loaded_imgs, usage, model_name, duration, batch_meta

def crop_images(pil_imgs, pages_data):
    crops = []
    if not os.path.exists(CROPS_DIR): os.makedirs(CROPS_DIR, exist_ok=True)
    
    for i, page_data in enumerate(pages_data):
        if i >= len(pil_imgs): src = pil_imgs[-1]
        else: src = pil_imgs[i]
        
        media_list = page_data.get("media_objects", [])
        w, h = src.size
        
        # Cover Guard
        is_cover_page = False
        cover_object = None
        for m in media_list:
            if m.get("type") == "cover" or m.get("id") == "book_cover":
                is_cover_page = True
                cover_object = m
                break
        
        if is_cover_page and cover_object:
            cover_object["coordinates"] = [0, 0, 1000, 1000]
            cover_object["id"] = "book_cover"
            media_list = [cover_object]
        
        for meta in media_list:
            try:
                ymin, xmin, ymax, xmax = meta["coordinates"]
                left = (xmin/1000)*w - CROP_PADDING
                top = (ymin/1000)*h - CROP_PADDING
                right = (xmax/1000)*w + CROP_PADDING
                bottom = (ymax/1000)*h + CROP_PADDING
                
                left = max(0, left); top = max(0, top)
                right = min(w, right); bottom = min(h, bottom)
                
                if (right-left < 30) or (bottom-top < 30): continue
                
                crop = src.crop((left, top, right, bottom))
                
                raw_id = meta.get("id", f"media_{int(time.time())}_{i}")
                if meta.get("type") == "cover": media_id = "book_cover"
                else: media_id = re.sub(r'[^a-zA-Z0-9_]', '', raw_id)
                
                c_name = f"{media_id}.png"
                c_path = os.path.join(CROPS_DIR, c_name)
                crop.save(c_path)
                
                caption = f"{meta.get('type', 'img').upper()}: {meta.get('caption_ru', '')}"
                crops.append((c_path, caption, media_id))
            except: pass
    return crops

# === TELEGRAM & UTILS ===
async def get_last_id(client, entity):
    try:
        full = await client(functions.channels.GetFullChannelRequest(entity))
        match = re.search(r"\[LastID:\s*(\d+)\]", full.full_chat.about or "")
        if match: return int(match.group(1))
    except: pass
    return None

async def update_last_id(client, entity, new_id):
    try:
        full = await client(functions.channels.GetFullChannelRequest(entity))
        about = full.full_chat.about or ""
        tag = f"[LastID: {new_id}]"
        if "[LastID:" in about: new_about = re.sub(r"\[LastID:\s*\d+\]", tag, about)
        else: new_about = f"{about}\n\n{tag}".strip()
        if new_about != about: await client(functions.messages.EditChatAboutRequest(peer=entity, about=new_about))
    except: pass

async def is_already_processed(message):
    try:
        if not message.reactions: return False
        for r in message.reactions.results:
            emoji = r.reaction.emoticon if hasattr(r.reaction, 'emoticon') else str(r.reaction)
            if emoji == MARKER_EMOJI and r.chosen: return True
    except: pass
    return False

async def mark_processed(client, msg):
    try:
        await client(functions.messages.SendReactionRequest(peer=msg.peer_id, msg_id=msg.id, reaction=[types.ReactionEmoji(emoticon=MARKER_EMOJI)]))
    except: pass

def render_pdf(path):
    doc = fitz.open(path)
    mat = fitz.Matrix(ZOOM_FACTOR, ZOOM_FACTOR)
    limit = len(doc)
    if DEBUG_PAGE_LIMIT:
        limit = min(len(doc), DEBUG_PAGE_LIMIT)
    for i in range(limit):
        page = doc.load_page(i)
        pix = page.get_pixmap(matrix=mat, alpha=False)
        out = os.path.join(TEMP_DIR, f"page_{i+1:04d}.png")
        pix.save(out)
        yield out, i+1, len(doc)
    doc.close()

def render_djvu(path):
    safe = os.path.join(TEMP_DIR, "temp.djvu")
    shutil.move(path, safe)
    try:
        subprocess.run([
            'ddjvu', '-format=tiff', f'-size={DJVU_TARGET_SIZE}x{DJVU_TARGET_SIZE}', 
            '-eachpage', safe, os.path.join(TEMP_DIR, "p_%04d.tif")
        ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE)
    except Exception as e:
        log(f"❌ DjVu Render Error: {e}")
        return

    files = sorted([f for f in os.listdir(TEMP_DIR) if f.endswith(".tif")])
    if DEBUG_PAGE_LIMIT: files = files[:DEBUG_PAGE_LIMIT]
    for i, f in enumerate(tqdm(files, desc="Conv", unit="pg", leave=False)):
        full = os.path.join(TEMP_DIR, f)
        try:
            with Image.open(full) as img:
                png = full.replace(".tif", ".png")
                img.save(png, "PNG")
                yield png, i+1, len(files)
            os.remove(full)
        except: pass
    if os.path.exists(safe): os.remove(safe)


# === OCR (TESSERACT) ===
async def run_ocr_for_pages(client, dest_entity, book_title, pages, global_resources=None):
    """
    Выполняет OCR для каждой страницы с помощью Tesseract и сохраняет
    JSON с массивом слов (text, bbox, conf) постранично.
    Затем отправляет JSON-файлы в Telegram батчами по 5 штук
    и регистрирует ссылки в global_resources.
    """
    if not pages or client is None or dest_entity is None:
        return

    base = os.path.splitext(book_title)[0]
    safe_base = re.sub(r"[^0-9A-Za-zА-Яа-яёЁ_-]+", "_", base)[:80]

    from pytesseract import Output

    json_items = []
    total_pages = len(pages)
    log(f"   🔍 OCR (Tesseract rus+deu+eng) for {total_pages} pages...")

    for local_idx, (img_path, page_num, total_in_doc) in enumerate(pages, start=1):
        try:
            log(f"      📝 OCR page {page_num}/{total_in_doc} ({local_idx}/{total_pages})...")
            img = Image.open(img_path)
            data = pytesseract.image_to_data(
                img,
                lang="rus+deu+eng",
                output_type=Output.DICT
            )

            words = []
            n = len(data.get("text", []))
            for i in range(n):
                text = (data["text"][i] or "").strip()
                if not text:
                    continue

                conf_raw = data.get("conf", ["0"])[i]
                try:
                    conf = float(conf_raw)
                except Exception:
                    conf = -1.0
                if conf < 0:
                    continue

                left = int(data.get("left", [0])[i])
                top = int(data.get("top", [0])[i])
                width = int(data.get("width", [0])[i])
                height = int(data.get("height", [0])[i])

                words.append({
                    "text": text,
                    "bbox": {"x": left, "y": top, "w": width, "h": height},
                    "conf": conf
                })

            payload = {
                "file_name": book_title,
                "safe_name": safe_base,
                "page_index": page_num,
                "page_in_doc": page_num,
                "total_pages": total_in_doc,
                "words": words
            }

            json_name = f"{safe_base}_p{page_num:04d}_words.json"
            json_path = os.path.join(OCR_JSON_DIR, json_name)
            with open(json_path, "w", encoding="utf-8") as f:
                json.dump(payload, f, ensure_ascii=False)

            json_items.append({
                "page_num": page_num,
                "total_pages": total_in_doc,
                "json_path": json_path,
                "json_name": json_name,
                "tg_msg_id": None
            })

        except Exception as e:
            log(f"      ⚠️ OCR error on page {page_num}: {str(e)[:100]}")
            continue

    if not json_items:
        log("   ⚠️ OCR produced no JSON files.")
        return

    # Отправляем батчами по 5 файлов
    BATCH_SIZE = 5
    for i in range(0, len(json_items), BATCH_SIZE):
        batch = json_items[i:i + BATCH_SIZE]
        try:
            batch_idx = i // BATCH_SIZE + 1
            log(f"   📤 Sending OCR JSON batch {batch_idx} ({len(batch)} files) to Telegram...")
            paths = [b["json_path"] for b in batch]
            caption = f"📄 OCR words [{safe_base}] batch {batch_idx}"
            result = await client.send_file(dest_entity, paths, caption=caption, force_document=True)

            # Telethon может вернуть одиночное сообщение или список сообщений
            if isinstance(result, (list, tuple)):
                msgs = list(result)
            else:
                msgs = [result]

            for meta, msg in zip(batch, msgs):
                meta["tg_msg_id"] = getattr(msg, "id", None)

        except Exception as e:
            log(f"   ⚠️ Failed to send OCR JSON batch: {str(e)[:120]}")
            continue

    # Добавляем OCR-ресурсы в global_resources, чтобы они попали в MD
    if global_resources is not None:
        for it in json_items:
            msg_id = it.get("tg_msg_id")
            if not msg_id:
                continue
            page_num = it["page_num"]
            res_id = f"ocr_{page_num:04d}_words"
            caption = f"OCR words page {page_num}"
            link = f"tg://msg_id?id={msg_id}"
            global_resources.append({"id": res_id, "caption": caption, "link": link})

    # Чистим временные JSON-файлы
    for it in json_items:
        try:
            os.remove(it["json_path"])
        except Exception:
            pass

# === MAIN ===
# === MAIN ===
async def main_logic():
    log(f"🚀 **Launch V57.1 (Full Meta + JSON Repair):** {datetime.now().strftime('%H:%M:%S')}")
    
    try:
        API_ID = int(user_secrets.get_secret("TELEGRAM_API_ID"))
        API_HASH = user_secrets.get_secret("TELEGRAM_API_HASH")
        BOT_TOKEN = user_secrets.get_secret("TELEGRAM_BOT_TOKEN")
        DEST_ID = int(user_secrets.get_secret("DEST_CHANNEL_ID"))
    except: return log("❌ Secrets missing.")

    for d in [TEMP_DIR, CROPS_DIR, OCR_JSON_DIR]:
        if not os.path.exists(d): os.makedirs(d, exist_ok=True)

    client = TelegramClient(MemorySession(), API_ID, API_HASH)
    await client.start(bot_token=BOT_TOKEN)
    log("✅ Telegram OK.")
    if not model_rotator: return await client.disconnect()

    try:
        src_ent = await client.get_entity(SOURCE_USERNAME)
        dst_ent = await client.get_entity(DEST_ID)
    except: return await client.disconnect()

    start_id = await get_last_id(client, src_ent) or 1
    curr_id = start_id + 1
    processed = 0
    empty_streak = 0
    t_in, t_out = 0, 0
    pbar = tqdm(total=SAFETY_LIMIT, desc="Scan", unit="msg")

    while processed < SAFETY_LIMIT:
        if empty_streak >= MAX_EMPTY_GAP: log("🏁 End."); break
        try:
            msgs = await client.get_messages(src_ent, ids=[curr_id])
            msg = msgs[0] if msgs else None
            
            if not msg or isinstance(msg, types.MessageEmpty):
                empty_streak += 1; curr_id += 1; pbar.update(1); continue
            
            if await is_already_processed(msg):
                await update_last_id(client, src_ent, msg.id)
                curr_id += 1; pbar.update(1); continue

            fname = getattr(msg.file, 'name', None)
            if not fname: curr_id += 1; pbar.update(1); continue
            ext = fname.split('.')[-1].lower()
            if ext not in ['pdf', 'djvu']: curr_id += 1; pbar.update(1); continue

            empty_streak = 0
            log(f"\n📘 **{fname}** (ID: {msg.id})")
            fpath = os.path.join(TEMP_DIR, fname)
            await msg.download_media(file=fpath)
            
            iter_pages = render_pdf(fpath) if ext == 'pdf' else render_djvu(fpath)
            
            # === FULL METADATA STRUCT V57 ===
            full_bibliographic_data = {
                "isbn": "",
                "title": "",
                "authors": [],
                "co_authors": [],
                "publisher": "",
                "year": "",
                "document_type": "",
                "historical_period_description": "",
                "historical_year_start": "",
                "historical_year_end": "",
                "summary": "",
                "source_telegram_channel_id": str(dst_ent.id),
                "source_origin_msg_id": str(msg.id)
            }
            
            global_text_blocks = []      
            global_resources = []        
            global_footnotes = []
            global_marginalia = set()
            
            last_batch_tail_text = None 
            current_batch_size = BATCH_STRATEGY[0]
            
            generated_pages_buffer = []
            for p in iter_pages: generated_pages_buffer.append(p)

            # === OCR STEP: word-level boxes to JSON ===
            try:
                await run_ocr_for_pages(client, dst_ent, fname, generated_pages_buffer, global_resources)
            except Exception as e:
                log(f"   ⚠️ OCR step failed: {str(e)[:120]}")

            
            cursor = 0
            total_pages = len(generated_pages_buffer)
            book_failed = False

            while cursor < total_pages:
                end = min(cursor + current_batch_size, total_pages)
                current_slice = generated_pages_buffer[cursor : end]
                slice_paths = [x[0] for x in current_slice]
                slice_idxs = [x[1] for x in current_slice]
                
                log(f"   🔄 Batch {slice_idxs[0]}-{slice_idxs[-1]} (Size: {len(slice_paths)})...")
                
                try:
                    pages_list, imgs, usg, m_name, duration, meta = await process_single_batch(
                        slice_paths, slice_idxs, last_batch_tail_text
                    )
                    
                    log(f"      ✅ [{m_name}] OK in {duration:.1f}s")
                    t_in += usg.get("in",0); t_out += usg.get("out",0)
                    
                    # === BATCH UPLOAD ===
                    page_msg_ids = {} 
                    ALBUM_CHUNK_SIZE = 10
                    for i in range(0, len(slice_paths), ALBUM_CHUNK_SIZE):
                        sub_paths = slice_paths[i : i + ALBUM_CHUNK_SIZE]
                        sub_idxs = slice_idxs[i : i + ALBUM_CHUNK_SIZE]
                        try:
                            sent_msgs = await client.send_file(dst_ent, file=sub_paths, force_document=True)
                            if not isinstance(sent_msgs, list): sent_msgs = [sent_msgs]
                            for sent_m, p_idx in zip(sent_msgs, sub_idxs):
                                page_msg_ids[p_idx] = sent_m.id
                            await asyncio.sleep(2) 
                        except: pass
                    # ====================

                    if meta and isinstance(meta, dict):
                        for k, v in meta.items():
                            if k in full_bibliographic_data and v:
                                # Overwrite empty fields, don't overwrite existing valid data if new one is partial
                                curr = full_bibliographic_data[k]
                                if not curr: full_bibliographic_data[k] = v

                    crops = crop_images(imgs, pages_list)
                    for cp_path, cp_cap, cp_id in crops:
                        msg_link = "local_error"
                        if os.path.exists(cp_path):
                            try:
                                c_msg = await client.send_file(dst_ent, cp_path, caption=f"🖼 {cp_cap}\n🆔 {cp_id}", force_document=False)
                                msg_link = f"tg://msg_id?id={c_msg.id}"
                            except: pass
                            finally:
                                if os.path.exists(cp_path): os.remove(cp_path)
                                await asyncio.sleep(2)
                        global_resources.append({"id": cp_id, "caption": cp_cap, "link": msg_link})

                    current_img_idx = 0
                    for i, page_data in enumerate(pages_list):
                        if i < len(slice_idxs): container_p = slice_idxs[i]
                        else: container_p = slice_idxs[-1]
                        
                        raw_text = page_data.get("main_text", "")
                        book_p = page_data.get("book_page_num", None)
                        class_code = page_data.get("classification_code", None)
                        
                        tg_ref_id = page_msg_ids.get(container_p, "N/A")
                        
                        if page_data.get("footnotes"): 
                            global_footnotes.append(f"[{container_p}] {page_data['footnotes']}")
                        
                        if page_data.get("marginalia"): 
                            cleaned_marg = page_data["marginalia"].strip()
                            if len(cleaned_marg) > 3: global_marginalia.add(cleaned_marg)
                        
                        final_page_text = raw_text.strip()
                        if class_code:
                            final_page_text = f"**{class_code}**\n\n" + final_page_text

                        global_text_blocks.append({
                            "container_idx": container_p,
                            "book_num": book_p,
                            "tg_ref": tg_ref_id, 
                            "text": final_page_text
                        })
                    
                    if global_text_blocks:
                        last_text_content = global_text_blocks[-1]["text"]
                        last_batch_tail_text = last_text_content[-1000:]
                    
                    for p in slice_paths:
                        if os.path.exists(p): os.remove(p)
                        
                    cursor += len(slice_paths)
                    
                except Exception as e:
                    log(f"      ⚠️ Failed: {str(e)[:100]}")
                    if "ResourceExhausted" in str(e) or "429" in str(e):
                        log("      ❌ CRITICAL API LIMIT. Stopping book.")
                        book_failed = True
                        break
                    
                    new_size = current_batch_size
                    for s in BATCH_STRATEGY:
                        if s < current_batch_size:
                            new_size = s
                            break
                    if new_size == current_batch_size:
                        cursor += 1 
                        current_batch_size = 1
                    else:
                        log(f"      ⬇️ Reducing batch size: {current_batch_size} -> {new_size}")
                        current_batch_size = new_size

            if book_failed:
                await client.send_message(dst_ent, f"❌ **Aborted:** {fname} (API Limits)")
                await update_last_id(client, src_ent, msg.id)
                continue

            # === GLOBAL ASSEMBLY ===
            final_md = f"# {fname}\n\n"
            
            final_md += "## Metadata\n"
            # Explicit order of ALL keys
            meta_order = [
                "isbn", "title", "authors", "co_authors", "publisher", "year", 
                "document_type", "historical_period_description", 
                "historical_year_start", "historical_year_end", "summary",
                "source_telegram_channel_id", "source_origin_msg_id"
            ]
            
            for k in meta_order:
                val = full_bibliographic_data.get(k, "")
                if isinstance(val, list):
                    val_str = ", ".join([str(x) for x in val if x]) # Filter empty strings
                else:
                    val_str = str(val) if val is not None else ""
                
                # Print key even if empty, but NO "N/A"
                final_md += f"- **{k}**: {val_str}\n"
            
            final_md += "\n## Content\n\n"


            # Build mapping: scan page index -> OCR JSON Telegram msg IDs (1..N per page)
            ocr_page_map = {}
            if global_resources:
                for res in global_resources:
                    rid = str(res.get("id") or "")
                    # We only care about OCR JSON entries like 'ocr_0001_words'
                    if rid.lower().startswith("ocr_"):
                        m_page = re.search(r"_(\d+)(?:_|$)", rid)
                        if not m_page:
                            continue
                        try:
                            page_idx = int(m_page.group(1))
                        except ValueError:
                            continue
                        link = str(res.get("link") or "")
                        m_id = re.search(r"id=(\d+)", link)
                        if not m_id:
                            continue
                        ocr_msg_id = m_id.group(1)
                        ocr_page_map.setdefault(page_idx, []).append(ocr_msg_id)

            joined_content = ""
            for i, block_data in enumerate(global_text_blocks):
                c_idx = block_data["container_idx"]
                b_num = block_data["book_num"]
                tg_id = block_data.get("tg_ref", "N/A")
                text = block_data["text"]

                # Optional OCR JSON ids for this scan page (take up to 2 just in case)
                ocr_list = ocr_page_map.get(int(c_idx)) if c_idx is not None else None
                ocr_suffix = ""
                if ocr_list:
                    ocr_suffix = " | OCRjson: " + ",".join(str(x) for x in ocr_list[:2])

                label = f"> `[File: {c_idx} | Ref: {tg_id}{ocr_suffix}]`"
                if b_num:
                    label = f"> `[File: {c_idx} | Book: {b_num} | Ref: {tg_id}{ocr_suffix}]`"

                page_marker = f"\n\n{label}\n"
                if i > 0: 
                    joined_content += page_marker + text
                else: 
                    joined_content += f"{label}\n" + text

            final_md += joined_content.strip()

            if global_resources:
                final_md += "\n\n## APPX: Resources\n"

                for res in global_resources:
                    final_md += f"- **ID**: `{res['id']}`\n"
                    final_md += f"  - **Caption**: {res['caption']}\n"
                    final_md += f"  - **Link**: {res['link']}\n"
            
            if global_footnotes:
                final_md += "\n\n## APPX: Footnotes\n" + "\n\n".join(global_footnotes)
            
            if global_marginalia:
                final_md += "\n\n## APPX: Marginalia\n" + "\n".join(sorted(list(global_marginalia)))

            md_out = os.path.join(TEMP_DIR, f"{os.path.splitext(fname)[0]}.md")
            with open(md_out, "w", encoding="utf-8") as f: f.write(final_md)
            await client.send_file(dst_ent, md_out, caption=f"📚 **MD FULL** (Tok: {t_in+t_out})", force_document=True)
            
            await mark_processed(client, msg)
            await update_last_id(client, src_ent, msg.id)
            log(f"   🎉 Done.")

        except Exception as e:
            log(f"❌ Error ID {curr_id}: {e}")
            import traceback; traceback.print_exc()
        
        finally:
            if os.path.exists(CROPS_DIR):
                shutil.rmtree(CROPS_DIR)
                os.makedirs(CROPS_DIR, exist_ok=True)
            if os.path.exists(TEMP_DIR):
                for f in os.listdir(TEMP_DIR):
                    if not f.endswith(".md"):
                        try: os.remove(os.path.join(TEMP_DIR, f))
                        except: pass
                        
        curr_id += 1; processed += 1; pbar.update(1)

    log(f"\n🏁 End. Total Tokens: {t_in}/{t_out}")
    await send_report(client, dst_ent)
    await client.disconnect()

# === RUN ===
print("🟡 Pre-flight check...")
try:
    _ = UserSecretsClient().get_secret("TELEGRAM_API_ID")
    print("🟢 Starting main_logic...")
    await main_logic()
except Exception as e: print(f"🔴 FAIL: {e}")

🧠 Models initialized. Main: models/gemini-2.5-flash. Delay: 30.0s.
🟡 Pre-flight check...
🟢 Starting main_logic...
🚀 **Launch V57.1 (Full Meta + JSON Repair):** 20:20:55
✅ Telegram OK.


Scan:   0%|          | 0/500 [00:00<?, ?msg/s]


📘 **Попов_М_,_Попадин_А_От_Канта_до_Твердого_соединения_далей,_к_ландшафту.pdf** (ID: 114)
   🔍 OCR (Tesseract rus+deu+eng) for 11 pages...
      📝 OCR page 1/18 (1/11)...
      📝 OCR page 2/18 (2/11)...
      📝 OCR page 3/18 (3/11)...
      📝 OCR page 4/18 (4/11)...
      📝 OCR page 5/18 (5/11)...
      📝 OCR page 6/18 (6/11)...
      📝 OCR page 7/18 (7/11)...
      📝 OCR page 8/18 (8/11)...
      📝 OCR page 9/18 (9/11)...
      📝 OCR page 10/18 (10/11)...
      📝 OCR page 11/18 (11/11)...
   📤 Sending OCR JSON batch 1 (5 files) to Telegram...
   📤 Sending OCR JSON batch 2 (5 files) to Telegram...
   📤 Sending OCR JSON batch 3 (1 files) to Telegram...
   🔄 Batch 1-5 (Size: 5)...
      📡 [models/gemini-2.5-flash] Sending...
      ✅ [models/gemini-2.5-flash] OK in 40.6s
   🔄 Batch 6-10 (Size: 5)...
      📡 [models/gemini-2.5-flash] Sending...
      ✅ [models/gemini-2.5-flash] OK in 17.3s
      ⚠️ Failed: 'NoneType' object has no attribute 'strip'
      ⬇️ Reducing batch size: 5 -> 3
  

In [4]:
# === ЯЧЕЙКА 3: V60.1 (FIXED CPU LOOP IN OVERLAP LOGIC) ===

import os
import time
import json
import asyncio
import glob
import re
import google.generativeai as genai
from datetime import datetime
from kaggle_secrets import UserSecretsClient
from telethon import TelegramClient
from telethon.sessions import MemorySession
import psycopg2
from psycopg2.extras import execute_values

# === SETTINGS ===
INPUT_DIR = "./temp_processing"
OUTPUT_PREVIEW_FILE = "RAG_DATA_PREVIEW.md"

# API Safety
GLOBAL_REQUEST_DELAY = 30.0  
TEXT_WINDOW_SIZE = 8000
WINDOW_OVERLAP = 1000  

# Chunk Constraints
MAX_CHARS_PER_CHUNK = 1400 

# Logic
RELEVANCE_THRESHOLD = 0.3 

# Debug Limit
BOOKS_LIMIT = 1 

# === AUTH SETUP ===
user_secrets = UserSecretsClient()
try:
    GENAI_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=GENAI_KEY)
    
    API_ID = int(user_secrets.get_secret("TELEGRAM_API_ID"))
    API_HASH = user_secrets.get_secret("TELEGRAM_API_HASH")
    BOT_TOKEN = user_secrets.get_secret("TELEGRAM_BOT_TOKEN")
    DEST_ID = int(user_secrets.get_secret("DEST_CHANNEL_ID"))
except:
    print("❌ Secrets missing")

MODEL_NAME = "models/gemini-2.5-flash"

# === UTILS ===
class RateLimiter:
    def __init__(self, delay):
        self.delay = delay
        self.last_call = 0
    
    async def wait(self):
        now = time.time()
        elapsed = now - self.last_call
        if elapsed < self.delay:
            wait_time = self.delay - elapsed + 1
            print(f"      ⏳ Pace: waiting {wait_time:.1f}s...")
            await asyncio.sleep(wait_time)
        self.last_call = time.time()

limiter = RateLimiter(GLOBAL_REQUEST_DELAY)

def get_flag_emoji(lang_code):
    flags = {
        "ru": "🇷🇺", "de": "🇩🇪", "en": "🇬🇧", 
        "pl": "🇵🇱", "fr": "🇫🇷", "it": "🇮🇹",
        "lt": "🇱🇹"
    }
    return flags.get(lang_code, "🌐")

def parse_processed_md(filepath):
    """
    Читает подготовленный MD-файл и возвращает:
    - meta: словарь метаданных книги
    - clean_text: основной текст без APPX-раздела
    - page_media: словарь {file_id(int) -> [{{"id","caption","link"}}]} для иллюстраций и прочих медиа
    - page_ocr: словарь {file_id(int) -> [{{"id","caption","link"}}]} для OCR JSON-файлов
    - page_scans: словарь {file_id(int) -> tg_msg_id(int)} для исходных сканов страниц (Ref)
    """
    if not os.path.exists(filepath):
        return None, None, {}, {}, {}, {}

    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    # --- 1. Metadata ---
    meta = {}
    meta_match = re.search(r'## Metadata\n(.*?)\n\n## Content', content, re.DOTALL)
    if meta_match:
        lines = meta_match.group(1).split('\n')
        for line in lines:
            if line.strip().startswith('- **'):
                match_row = re.search(r'\*\*(\w+)\*\*: (.*)', line)
                if match_row:
                    key = match_row.group(1)
                    val = match_row.group(2).strip()
                    if val.lower() != "null" and val != "":
                        meta[key] = val

    # --- 2. Main text (without APPX) ---
    text_match = re.search(r'## Content\n(.*?)(\n## APPX|$)', content, re.DOTALL)
    clean_text = text_match.group(1).strip() if text_match else ""

    # --- 3. APPX: Resources -> page_media + page_ocr ---
    page_media = {}
    page_ocr = {}
    appx_match = re.search(r'## APPX: Resources\n(.*)', content, re.DOTALL)
    if appx_match:
        lines = appx_match.group(1).split('\n')
        current = None
        for line in lines:
            line = line.rstrip()
            if not line:
                continue

            m_id = re.match(r"- \*\*ID\*\*: `([^`]+)`", line)
            if m_id:
                # Финализируем предыдущую запись
                if current and current.get("id"):
                    mid = current["id"]
                    m_page = re.search(r'_(\d+)(?:_|$)', mid)
                    if m_page:
                        try:
                            page = int(m_page.group(1))
                        except ValueError:
                            page = None
                        if page is not None:
                            # Распределяем по media/ocr в зависимости от id
                            if mid.lower().startswith("ocr_"):
                                page_ocr.setdefault(page, []).append(current)
                            else:
                                page_media.setdefault(page, []).append(current)
                current = {"id": m_id.group(1)}
                continue

            m_cap = re.match(r"- \*\*Caption\*\*: (.*)", line)
            if m_cap and current is not None:
                current["caption"] = m_cap.group(1).strip()
                continue

            m_link = re.match(r"- \*\*Link\*\*: (.*)", line)
            if m_link and current is not None:
                current["link"] = m_link.group(1).strip()
                # Финализируем запись сразу после Link
                mid = current.get("id")
                if mid:
                    m_page = re.search(r'_(\d+)(?:_|$)', mid)
                    if m_page:
                        try:
                            page = int(m_page.group(1))
                        except ValueError:
                            page = None
                        if page is not None:
                            if mid.lower().startswith("ocr_"):
                                page_ocr.setdefault(page, []).append(current)
                            else:
                                page_media.setdefault(page, []).append(current)
                current = None

        # На всякий случай финализируем последнюю запись без Link
        if current and current.get("id"):
            mid = current["id"]
            m_page = re.search(r'_(\d+)(?:_|$)', mid)
            if m_page:
                try:
                    page = int(m_page.group(1))
                except ValueError:
                    page = None
                if page is not None:
                    if mid.lower().startswith("ocr_"):
                        page_ocr.setdefault(page, []).append(current)
                    else:
                        page_media.setdefault(page, []).append(current)

    # --- 4. Page scans (File: N | Ref: tg_id, OCRjson: ids) ---
    page_scans = {}
    page_ocr_refs = {}
    # Ищем маркеры вида [File: 3 | Book: ... | Ref: 12345] или [File: 3 | Ref: 12345]
    # Теперь маркер может дополнительно содержать хвост вида "| OCRjson: 1415,1416"
    pattern_scans = r'\[File:\s*(\d+)(?:\s*\|\s*Book:[^\]|]*)?\s*\|\s*Ref:\s*(\d+)(?:\s*\|\s*OCRjson:\s*([\d,\s]+))?\]'
    for m in re.finditer(pattern_scans, content):
        try:
            fid = int(m.group(1))
            tg_id = int(m.group(2))
            page_scans[fid] = tg_id

            ocr_raw = m.group(3)
            if ocr_raw:
                ids = []
                for part in re.split(r'[,\s]+', ocr_raw.strip()):
                    if not part:
                        continue
                    try:
                        ids.append(int(part))
                    except ValueError:
                        continue
                if ids:
                    existing = page_ocr_refs.get(fid, [])
                    for vid in ids:
                        if vid not in existing:
                            existing.append(vid)
                    page_ocr_refs[fid] = existing
        except ValueError:
            continue

    return meta, clean_text, page_media, page_ocr, page_scans, page_ocr_refs



def create_windows_with_overlap(text, window_size, overlap):
    """
    Безопасная нарезка окон с нахлестом.
    Гарантирует отсутствие бесконечных циклов.
    """
    paragraphs = text.split('\n\n')
    windows = []
    
    current_window = []
    current_len = 0
    
    for para in paragraphs:
        para_len = len(para)
        
        # Если добавление параграфа превысит лимит окна
        if current_len + para_len > window_size:
            # 1. Если в окне уже что-то есть, сохраняем его как готовое окно
            if current_window:
                windows.append("\n\n".join(current_window))
                
                # 2. Вычисляем Overlap (хвост текущего окна станет головой следующего)
                # Идем назад, пока не наберем нужное количество символов
                back_buffer = []
                back_len = 0
                for p in reversed(current_window):
                    if back_len >= overlap: break
                    back_buffer.insert(0, p)
                    back_len += len(p)
                
                # 3. Новое окно начинается с Overlap
                current_window = back_buffer
                current_len = back_len
        
        # 4. Добавляем текущий параграф (безусловно добавляем, чтобы двигаться вперед)
        current_window.append(para)
        current_len += para_len
        
    # Не забываем хвост
    if current_window:
        windows.append("\n\n".join(current_window))
        
    return windows

def get_chunking_prompt(book_meta, text_window, prev_context):
    title = book_meta.get('title', 'Unknown Title')

    # Where in the document we are (for locators)
    loc_str = "Start of document."
    if prev_context.get("file_id", 1) != 1 or prev_context.get("tg_msg_id", 0) != 0:
        loc_str = f"[File: {prev_context.get('file_id', 1)} | Book: {prev_context.get('book_page', '1')} | Ref: {prev_context.get('tg_msg_id', 0)}]"

    # Short context from the last accepted content chunk
    last_text_context = "No previous text."
    if prev_context.get("last_chunk_text"):
        safe_text = prev_context['last_chunk_text'].replace('"', "'")
        last_text_context = f"\"{safe_text}\""

    return f"""
    ROLE: Database Engineer and RAG chunker.
    TASK: Split text into SMALL, COHERENT SEMANTIC CHUNKS (for vector search).

    DOCUMENT INFO:
    Title: "{title}"

    CONTEXT FROM PREVIOUS BATCH:
    - Last Location: {loc_str}
    - LAST CHUNK CONTENT (use this to fix broken sentences):
      {last_text_context}

    INPUT TEXT (contains ~{WINDOW_OVERLAP} chars overlap from previous batch):
    \"""{text_window}\""" 

    INSTRUCTIONS:
    1. ENTITY TYPES:
       - Use `chunk_type` = "content" for the main narrative, analysis, introductions, conclusions, and abstracts (даже если они на английском).
       - Use `chunk_type` = "bibliography" for lists of literature, references and source lists.
       - Use `chunk_type` = "footnotes_definitions" for footnotes, archival references, пояснения к ссылкам.

    2. CHUNK SIZE & SENTENCE INTEGRITY:
       - TARGET SIZE: Aim for 500–900 characters per `chunk_content`.
       - HARD LIMIT: Each `chunk_content` MUST be < {MAX_CHARS_PER_CHUNK} characters.
       - NEVER cut a sentence between chunks. Each `chunk_content` should end at a sentence boundary (. ? ! … or closing quotes), except for very short headings.
       - If the previous `LAST CHUNK CONTENT` ended in the middle of a sentence, use the overlapped text to restore the FULL sentence inside a single chunk.
       - If you must choose between slightly exceeding the target and breaking a sentence, prefer a slightly longer chunk (but still under the HARD LIMIT).

    3. OVERLAP & CONTINUITY:
       - The `INPUT TEXT` starts with text from the previous batch (overlap of about {WINDOW_OVERLAP} characters).
       - DO NOT DUPLICATE: do not output text that is largely identical to `LAST CHUNK CONTENT`.
       - The first chunk in this batch should smoothly continue after `LAST CHUNK CONTENT`, without repeating it.

    4. LOCATOR EXTRACTION:
       - For every chunk, find the NEAREST PRECEDING marker `> [File: X | Book: Y | Ref: Z]` in the original text.
       - If no marker is present inside this batch (because it is only in the overlap), USE the `Last Location` from the context instead.
       - Fill `file_id`, `book_page`, `tg_msg_id` based on this nearest marker / context.
       - If a chunk clearly spans TWO neighbouring pages (text continues after the next marker),
         also fill `file_id_2`, `book_page_2`, `tg_msg_id_2` for the second page; otherwise set them to null.

    5. LANGUAGE & METADATA:
       - Field `lang`: two-letter language code such as "ru", "de", "en", "pl", etc.
       - Fields `semantic_header` and `keywords` MUST be written in Russian, even if `chunk_content` is in another language.
       - EXCEPTION: if `lang` == "de" (German text), write `semantic_header` and `keywords` in German.
       - For English abstracts, summaries and keyword blocks that describe a Russian article, STILL produce a Russian `semantic_header` and Russian `keywords` that accurately convey the meaning.
       - `semantic_header` should be 1 short sentence or phrase summarising the chunk.
       - `keywords` should be a short list (3–8) of topical tags in Russian.

    6. RELEVANCE SCORE (0.0–1.0):
       - 0.9–1.0: Core analytical or narrative content that is central to the document’s main topic.
       - 0.6–0.8: Important supporting text: introductions, abstracts (including foreign-language abstracts), conclusions, author commentary.
       - 0.3–0.5: Contextual but secondary material (less central details, local digressions).
       - 0.1–0.2: Mostly technical or marginal text: isolated archival references, very noisy OCR, marginalia.
       - 0.0: Only for garbage (pure OCR noise, meaningless fragments, empty chunks).
       - Abstracts, summaries and keyword blocks are NEVER garbage: do NOT assign them 0.0 unless the text is unreadable.
       - Bibliography and footnotes can have lower scores (e.g. 0.2–0.5), but should still honestly reflect their usefulness.

    7. PERSONS & LOCATIONS (for metadata filters):
    7. PERSONS & LOCATIONS (for metadata filters):
    - For each chunk, also extract two additional fields:
      - `persons`: a list of real people (full names where possible — philosophers, rulers, clergy, authors, etc.) explicitly mentioned in `chunk_content`. Do NOT include generic roles like "king" or "author" without names.
      - `locations`: a list of geographic or topographical entities (cities, towns, villages, regions, rivers, churches, monasteries, streets, districts, buildings, landmarks) explicitly mentioned in `chunk_content` or clearly and unambiguously implied by it (for example, "Кафедральный собор" in Königsberg).
    - If there are no persons or locations, return empty arrays: `"persons": []`, `"locations": []` (do NOT use null).
    - PRIORITIZE MICRO-LOCATIONS: when possible, focus on specific landmarks, buildings, streets, quarters and districts (e.g., "Башня Дона", "Озеро Верхнее", "Hotel Sambia", "Дом Сказочника") instead of only broad city names.
    - INFER TOWN FROM LANDMARK (Kaliningrad region): if a landmark unambiguously belongs to a town in the Kaliningrad region, you may include both the object itself and the town name as separate location entries. Example: "Hotel Sambia" → locations: ["Hotel Sambia", "Зеленоградск"]; "Дом Сказочника" → locations: ["Дом Сказочника", "Светлогорск"].
    - AVOID OVER-GENERIC KALININGRAD: do not add "Калининград" or "Калининградская область" to `locations` if more specific locations (streets, buildings, landmarks) inside the city or region are already listed. Keep "Калининград" only if the text discusses the city as a whole (e.g., its history, administration, population) rather than a particular point of interest.

    8. LANGUAGE & ORTHOGRAPHY (CRITICAL):
       - Preserve the original spelling in `chunk_content` (keep 'ѣ', 'ъ', 'i' etc.). Do NOT modernize the visible text.
       - NORMALIZATION FOR METADATA: All lists (`keywords`, `persons`, `locations`) MUST be converted to Modern Russian spelling.
         * Example: text says "Кенигсбергѣ" → location tag must be "Кенигсберг".
         * Example: text says "Фордере Форштадтъ" → location tag must be "Фордере Форштадт".
       - NORMALIZATION FOR SEARCH: Populate the `normalized_content` field with the full chunk text converted to modern Russian spelling.
       - If the text uses pre-1918 orthography, set `"orthography": "pre_reform"`. Otherwise use `"orthography": "modern"`.


    
    OUTPUT JSON FORMAT (Array):
    [
      {{
        "chunk_type": "content" | "bibliography" | "footnotes_definitions",
        "chunk_content": "string (ORIGINAL text with ѣ, ъ, i - preserve exactly as in source)",
        "normalized_content": "string (Modern Russian spelling for search indexing)",
        "file_id": int,
        "book_page": "string",
        "tg_msg_id": int,
        "file_id_2": int | null,
        "book_page_2": "string" | null,
        "tg_msg_id_2": int | null,
        "relevance_score": float,
        "lang": "code",
        "semantic_header": "string",
        "keywords": ["tag1", "tag2", "tag3"],
        "persons": ["Иммануил Кант", "Герцог Альбрехт"],
        "locations": ["Кёнигсберг", "Кафедральный собор"],
        "orthography": "modern" | "pre_reform"
      }}
    ]
    """

# === SUPABASE / POSTGRES RAG STORAGE ===
def db_log(msg: str):
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[DB] {ts} {msg}")

def get_db_url():
    url = os.getenv("SUPABASE_DB_URL")
    if not url:
        try:
            user_secrets = UserSecretsClient()
            url = user_secrets.get_secret("SUPABASE_DB_URL")
        except Exception:
            url = None
    if not url:
        raise RuntimeError("SUPABASE_DB_URL not found. Set SUPABASE_DB_URL env or Kaggle secret.")
    return url

def ensure_db_schema():
    """
    Идемпотентно проверяет и создаёт расширения и таблицы:
    books, media, chunks (без таблицы pages).
    """
    db_url = get_db_url()
    db_log("Connecting to database for schema check...")
    conn = psycopg2.connect(db_url)
    conn.autocommit = True
    cur = conn.cursor()

    # Extensions
    db_log("Checking extensions (vector, pg_trgm, pgcrypto)...")
    cur.execute("create extension if not exists vector;")
    cur.execute("create extension if not exists pg_trgm;")
    cur.execute("create extension if not exists pgcrypto;")

    # books
    db_log("Checking table books...")
    cur.execute("select to_regclass('public.books');")
    if cur.fetchone()[0] is None:
        db_log("Creating table books...")
        cur.execute("""create table public.books (
            id uuid primary key default gen_random_uuid(),
            isbn text unique,
            title text not null,
            subtitle text,
            authors text,
            co_authors text,
            publisher text,
            year int,
            document_type text,
            historical_period_description text,
            historical_year_start int,
            historical_year_end int,
            summary text,
            source_telegram_channel_id bigint,
            source_origin_msg_id bigint,
            source_origin_file text,
            meta jsonb,
            created_at timestamptz default now(),
            updated_at timestamptz default now()
        );""")
        cur.execute("create index idx_books_title_trgm on public.books using gin (title gin_trgm_ops);")
        cur.execute("create index idx_books_year on public.books(year);")
        db_log("Table books created.")
    else:
        db_log("Table books already exists.")

    # media
    db_log("Checking table media...")
    cur.execute("select to_regclass('public.media');")
    if cur.fetchone()[0] is None:
        db_log("Creating table media...")
        cur.execute("""create table public.media (
            id bigserial primary key,
            book_id uuid references public.books(id) on delete cascade,
            internal_id text,
            tg_msg_id bigint,
            file_id int,
            book_page text,
            caption text,
            meta jsonb,
            created_at timestamptz default now()
        );""")
        cur.execute("create index idx_media_book on public.media(book_id);")
        db_log("Table media created.")
    else:
        db_log("Table media already exists.")

    # chunks
    db_log("Checking table chunks...")
    cur.execute("select to_regclass('public.chunks');")
    if cur.fetchone()[0] is None:
        db_log("Creating table chunks...")
        cur.execute("""create table public.chunks (
            id bigserial primary key,
            book_id uuid references public.books(id) on delete cascade,
            file_id int,
            book_page text,
            tg_msg_id bigint,
            position int,
            chunk_type text,
            lang text,
            relevance_score real,
            topic text,
            keywords text[],
            persons text[],
            locations text[],
            orthography text,
            chunk_year_start int,
            chunk_year_end int,
            scan_page_ids int[],
            scan_tg_msg_ids bigint[],
            ocr_tg_msg_ids bigint[],
            media_ids bigint[],
            media_internal_ids text[],
            content text,
            embedding vector(768),
            created_at timestamptz default now()
        );""")
        cur.execute("create index idx_chunks_book on public.chunks(book_id);")
        cur.execute("create index idx_chunks_relevance on public.chunks(relevance_score);")
        cur.execute("create index idx_chunks_embedding on public.chunks using hnsw (embedding vector_cosine_ops);")
        db_log("Table chunks created.")
    else:
        db_log("Table chunks already exists.")
        cur.execute("alter table public.chunks add column if not exists persons text[];")
        cur.execute("alter table public.chunks add column if not exists locations text[];")
        cur.execute("alter table public.chunks add column if not exists orthography text;")
        cur.execute("alter table public.chunks add column if not exists scan_page_ids int[];")
        cur.execute("alter table public.chunks add column if not exists scan_tg_msg_ids bigint[];")
        cur.execute("alter table public.chunks add column if not exists ocr_tg_msg_ids bigint[];")


        # === CREATE / UPDATE VECTOR SEARCH FUNCTION ===
    db_log("Checking/Updating match_chunks function...")

    # 1. ДОБАВИТЬ ЭТУ СТРОКУ (удаление старой версии функции):
    cur.execute("drop function if exists match_chunks(vector(768), float, int);")

    cur.execute("""
    create or replace function match_chunks (
      query_embedding vector(768),
      match_threshold float,
      match_count int
    )
    returns table (
      -- (тут без изменений список возвращаемых полей)
      id int8,
      book_id uuid,
      content text,
      similarity float,
      chunk_type text,
      lang text,
      relevance_score real,
      topic text,
      keywords text[],
      persons text[],
      locations text[],
      orthography text,
      chunk_year_start int,
      chunk_year_end int,
      scan_page_ids int[],
      scan_tg_msg_ids bigint[],
      ocr_tg_msg_ids bigint[],
      media_ids bigint[],
      media_internal_ids text[],
      tg_msg_id bigint,
      book_page text,
      book_title text,
      book_authors text,
      book_year int,
      book_isbn text,
      source_links text[]
    )
    language plpgsql
    as $$
    begin
      return query
      select
        c.id,
        c.book_id,
        c.content,
        1 - (c.embedding <=> query_embedding) as similarity,
        c.chunk_type,
        c.lang,
        c.relevance_score,
        c.topic,
        c.keywords,
        c.persons,
        c.locations,
        c.orthography,
        c.chunk_year_start,
        c.chunk_year_end,
        c.scan_page_ids,
        c.scan_tg_msg_ids,
        c.ocr_tg_msg_ids,
        c.media_ids,
        c.media_internal_ids,
        c.tg_msg_id,
        c.book_page,
        b.title as book_title,
        b.authors as book_authors,
        b.year as book_year,
        b.isbn as book_isbn,
        case
          when c.scan_tg_msg_ids is null or array_length(c.scan_tg_msg_ids, 1) = 0 then
            array[
              'https://t.me/c/' || b.source_telegram_channel_id::text || '/' || c.tg_msg_id::text
            ]
          else (
            -- 2. ИСПРАВЛЕНИЕ ЗДЕСЬ (scan_id вместо id):
            select array_agg(
              'https://t.me/c/' || b.source_telegram_channel_id::text || '/' || scan_id::text
            )
            from unnest(c.scan_tg_msg_ids) as scan_id
          )
        end as source_links
      from chunks c
      left join books b on c.book_id = b.id
      where 1 - (c.embedding <=> query_embedding) > match_threshold
      order by c.embedding <=> query_embedding
      limit match_count;
    end;
    $$;
    """)

    cur.close()
    conn.close()
    db_log("Schema check complete.")

def find_existing_book(conn, meta: dict):
    """Возвращает id книги, если в таблице books уже есть запись с таким же ISBN
    или (title + year). Ничего не удаляет и не обновляет."""
    cur = conn.cursor()
    isbn = (meta.get("isbn") or "").strip() or None
    title = (meta.get("title") or "").strip()
    year = meta.get("year")

    if isbn:
        cur.execute("select id from public.books where isbn = %s;", (isbn,))
        row = cur.fetchone()
        if row:
            db_log(f"Book with same ISBN already exists: {isbn}")
            return row[0]

    if title and year:
        cur.execute("select id from public.books where title = %s and year = %s limit 1;", (title, year))
        row = cur.fetchone()
        if row:
            db_log(f"Book with same title+year already exists: {title} ({year})")
            return row[0]

    return None

def insert_book_and_related(book_data: dict):
    """Вставляет одну книгу и все её чанки/медиа в БД Supabase.
    НИЧЕГО не удаляет и не обновляет, если книга уже есть (isbn или title+year).
    Также аккуратно приводит media_ids к bigint[], игнорируя нечисловые значения.
    """
    db_url = get_db_url()
    ensure_db_schema()
    conn = psycopg2.connect(db_url)
    conn.autocommit = False
    cur = conn.cursor()

    meta = book_data.get("metadata", {})
    page_ocr_map = book_data.get("page_ocr") or {}

    def extract_tg_msg_id_from_link(link: str):
        """Извлекает числовой msg_id из строки вида 'tg://msg_id?id=1007'."""
        if not link:
            return None
        m = re.search(r"id=(\d+)", link)
        if not m:
            return None
        try:
            return int(m.group(1))
        except ValueError:
            return None


    existing_id = find_existing_book(conn, meta)
    if existing_id:
        db_log(f"Skip insert: book already exists in DB (id={existing_id}).")
        conn.close()
        return

    db_log("Inserting new book row...")
    cur.execute(
        """insert into public.books(
            isbn, title, subtitle, authors, co_authors, publisher, year,
            document_type, historical_period_description,
            historical_year_start, historical_year_end,
            summary, source_telegram_channel_id,
            source_origin_msg_id, source_origin_file, meta
        ) values (
            %(isbn)s, %(title)s, %(subtitle)s, %(authors)s, %(co_authors)s, %(publisher)s, %(year)s,
            %(document_type)s, %(historical_period_description)s,
            %(historical_year_start)s, %(historical_year_end)s,
            %(summary)s, %(source_telegram_channel_id)s,
            %(source_origin_msg_id)s, %(source_origin_file)s, %(meta)s
        ) returning id;""",
        {
            "isbn": meta.get("isbn"),
            "title": meta.get("title"),
            "subtitle": meta.get("subtitle"),
            "authors": meta.get("authors"),
            "co_authors": meta.get("co_authors"),
            "publisher": meta.get("publisher"),
            "year": meta.get("year"),
            "document_type": meta.get("document_type"),
            "historical_period_description": meta.get("historical_period_description"),
            "historical_year_start": meta.get("historical_year_start"),
            "historical_year_end": meta.get("historical_year_end"),
            "summary": meta.get("summary"),
            "source_telegram_channel_id": meta.get("source_telegram_channel_id"),
            "source_origin_msg_id": meta.get("source_origin_msg_id"),
            "source_origin_file": meta.get("source_origin_file"),
            "meta": json.dumps(meta, ensure_ascii=False),
        },
    )
    book_id = cur.fetchone()[0]
    db_log(f"Book inserted with id={book_id}")

    # MEDIA: берём page_media как есть
    page_media = book_data.get("page_media") or {}
    media_rows = []
    for page_key, media_list in page_media.items():
        try:
            file_id = int(page_key)
        except (TypeError, ValueError):
            file_id = None
        for m in media_list:
            internal_id = m.get("id")
            caption = m.get("caption")
            link = m.get("link") or ""
            tg_msg_id = None
            m_id = re.search(r"id=(\d+)", link)
            if m_id:
                try:
                    tg_msg_id = int(m_id.group(1))
                except ValueError:
                    tg_msg_id = None
            media_rows.append(
                (
                    book_id,
                    internal_id,
                    tg_msg_id,
                    file_id,
                    None,
                    caption,
                    json.dumps(m, ensure_ascii=False),
                )
            )

    if media_rows:
        db_log(f"Inserting {len(media_rows)} media rows...")
        execute_values(
            cur,
            "insert into public.media (book_id, internal_id, tg_msg_id, file_id, book_page, caption, meta) values %s",
            media_rows,
        )

    # CHUNKS
    all_chunks = []
    for section_name in ("chunks", "bibliography", "footnotes"):
        for ch in book_data.get(section_name, []):
            all_chunks.append(ch)

    def embed_one(text: str):
        """Строит эмбеддинг для одного текста. При ошибке возвращает None."""
        try:
            if not text:
                text = ""
            result = genai.embed_content(model="text-embedding-004", content=text)
            if isinstance(result, dict) and "embedding" in result:
                return result["embedding"]
            return result
        except Exception as e:
            db_log(f"Embedding error: {e}")
            return None

    chunk_rows = []
    for pos, ch in enumerate(all_chunks):
        original_text = ch.get("chunk_content") or ""
        normalized_text = ch.get("normalized_content") or original_text
        text = normalized_text

        # prepare metadata for enriched embedding
        keywords = ch.get("keywords") or []
        if isinstance(keywords, str):
            keywords = [k.strip() for k in keywords.split(",") if k.strip()]

        persons = ch.get("persons") or []
        if isinstance(persons, str):
            persons = [p.strip() for p in persons.split(",") if p.strip()]

        locations = ch.get("locations") or []
        if isinstance(locations, str):
            locations = [l.strip() for l in locations.split(",") if l.strip()]

        orthography = ch.get("orthography") or None

        semantic_header = ch.get("semantic_header") or ""

        rich_parts = []
        if semantic_header:
            rich_parts.append(f"Тема: {semantic_header}")
        if keywords:
            rich_parts.append("Ключевые слова: " + ", ".join(keywords))
        if persons:
            rich_parts.append("Персоны: " + ", ".join(persons))
        if locations:
            rich_parts.append("Локации: " + ", ".join(locations))
        if text:
            rich_parts.append("Текст: " + text)
        rich_context = "\n".join(rich_parts)

        emb = embed_one(rich_context)

        # file_id / book_page / tg_msg_id
        try:
            fid_int = int(ch.get("file_id")) if ch.get("file_id") is not None else None
        except (TypeError, ValueError):
            fid_int = None

        book_page = str(ch.get("book_page")) if ch.get("book_page") is not None else None
        tg_msg_id = ch.get("tg_msg_id")

        # keywords
        keywords = ch.get("keywords") or []
        if isinstance(keywords, str):
            keywords = [k.strip() for k in keywords.split(",") if k.strip()]

        # media_ids: приводим к списку bigint, а все нечисловые добавляем к media_internal_ids
        raw_media_ids = ch.get("media_ids") or []
        if not isinstance(raw_media_ids, (list, tuple)):
            raw_media_ids = [raw_media_ids]

        cast_media_ids = []
        extra_internal_ids = []
        for x in raw_media_ids:
            if x is None:
                continue
            # сначала пробуем как число
            if isinstance(x, (int, float)):
                cast_media_ids.append(int(x))
                continue
            s = str(x).strip()
            if not s:
                continue
            try:
                cast_media_ids.append(int(s))
            except ValueError:
                extra_internal_ids.append(s)

        # media_internal_ids исходные + добавленные из "битых" media_ids
        internal_ids = ch.get("media_internal_ids") or []
        if not isinstance(internal_ids, (list, tuple)):
            internal_ids = [internal_ids]
        internal_ids = list(internal_ids) + extra_internal_ids

        # Приводим scan_page_ids и scan_tg_msg_ids к аккуратному виду
        scan_page_ids = ch.get("scan_page_ids") or None
        if isinstance(scan_page_ids, int):
            scan_page_ids = [scan_page_ids]
        if isinstance(scan_page_ids, (list, tuple)):
            scan_page_ids = [int(p) for p in scan_page_ids if p is not None]
            if not scan_page_ids:
                scan_page_ids = None

        scan_tg_ids = ch.get("scan_tg_msg_ids") or None
        if isinstance(scan_tg_ids, (int, float)):
            scan_tg_ids = [int(scan_tg_ids)]
        if isinstance(scan_tg_ids, (list, tuple)):
            scan_tg_ids = [int(t) for t in scan_tg_ids if t is not None]
            if not scan_tg_ids:
                scan_tg_ids = None

        # OCR JSON ids: используем то, что уже прикреплено к чанку,
        # а если пусто — восстанавливаем по page_ocr_map и scan_page_ids
        ocr_ids = ch.get("ocr_tg_msg_ids") or None
        if (not ocr_ids) and scan_page_ids and page_ocr_map:
            tmp = []
            for pid in scan_page_ids:
                page_ocr_list = page_ocr_map.get(pid) or []
                for mobj in page_ocr_list:
                    msg_id = extract_tg_msg_id_from_link(mobj.get("link", ""))
                    if msg_id is not None and msg_id not in tmp:
                        tmp.append(msg_id)
            if tmp:
                ocr_ids = tmp

        if isinstance(ocr_ids, (int, float)):
            ocr_ids = [int(ocr_ids)]
        if isinstance(ocr_ids, (list, tuple)):
            ocr_ids = [int(t) for t in ocr_ids if t is not None]
            if not ocr_ids:
                ocr_ids = None

        chunk_rows.append(
            (
                book_id,
                fid_int,
                book_page,
                tg_msg_id,
                pos,
                ch.get("chunk_type") or "content",
                ch.get("lang"),
                float(ch.get("relevance_score") or 0.0),
                ch.get("semantic_header"),
                keywords,
                persons,
                locations,
                orthography,
                ch.get("chunk_year_start"),
                ch.get("chunk_year_end"),
                scan_page_ids,
                scan_tg_ids,
                ocr_ids,
                cast_media_ids or None,
                internal_ids or None,
                text,
                emb,
            )
        )


    if chunk_rows:
        db_log(f"Inserting {len(chunk_rows)} chunks...")
        execute_values(
            cur,
            """insert into public.chunks (
                book_id, file_id, book_page, tg_msg_id, position,
                chunk_type, lang, relevance_score, topic, keywords,
                persons, locations, orthography,
                chunk_year_start, chunk_year_end,
                scan_page_ids, scan_tg_msg_ids, ocr_tg_msg_ids,
                media_ids, media_internal_ids,
                content, embedding
            ) values %s""",
            chunk_rows,
        )

    conn.commit()
    cur.close()
    conn.close()
    db_log("Book data committed to DB.")

async def process_rag_pipeline():
    # Ensure DB schema is ready (idempotent)
    try:
        ensure_db_schema()
    except Exception as e:
        print(f"[DB] ⚠️ Schema check failed: {e}")
    all_md_files = sorted(glob.glob(os.path.join(INPUT_DIR, "*.md")))
    valid_files = [f for f in all_md_files if "RAG_DATA" not in f]
    
    if not valid_files:
        print("❌ No source MD files found.")
        return

    files_to_process = valid_files[:BOOKS_LIMIT]
    print(f"📂 Found {len(valid_files)}. Processing: {len(files_to_process)}.")

    # Build unique preview filename based on first source file and current time
    run_ts = datetime.now().strftime("%Y%m%d_%H%M")
    first_base = os.path.splitext(os.path.basename(files_to_process[0]))[0]
    safe_base = re.sub(r"[^0-9A-Za-zА-Яа-я]+", "_", first_base).strip("_")
    if len(files_to_process) == 1:
        output_preview_file = f"RAG_DATA_PREVIEW_{safe_base}_{run_ts}.md"
    else:
        output_preview_file = f"RAG_DATA_PREVIEW_BATCH_{run_ts}.md"

    
    total_in, total_out = 0, 0
    
    client = TelegramClient(MemorySession(), API_ID, API_HASH)
    await client.start(bot_token=BOT_TOKEN)
    dest_ent = await client.get_entity(DEST_ID)
    
    preview_md = f"# RAG DATA PREVIEW (V60.1 Fixed)\nDate: {datetime.now().strftime('%Y-%m-%d %H:%M')}\n\n"
    
    for md_path in files_to_process:
        filename = os.path.basename(md_path)
        print(f"\n📘 Processing: {filename}")
        
        meta, full_text, page_media, page_ocr, page_scans, page_ocr_refs = parse_processed_md(md_path)
        if not full_text: continue

        book_data = {
            "metadata": meta,
            "chunks": [],
            "bibliography": [],
            "footnotes": [],
            "page_media": page_media or {},
            "page_ocr": page_ocr or {},
            "page_scans": page_scans or {},
            "page_ocr_refs": page_ocr_refs or {}
        }
        # USE SAFE OVERLAP FUNCTION
        windows = create_windows_with_overlap(full_text, TEXT_WINDOW_SIZE, WINDOW_OVERLAP)
        print(f"   ✂️ Windows: {len(windows)} (Overlap: {WINDOW_OVERLAP})")
        
        context_memory = {
            "file_id": 1,        
            "book_page": "1",    
            "tg_msg_id": 0,
            "last_chunk_text": None
        }
        
        for i, window in enumerate(windows):
            await limiter.wait()
            
            model = genai.GenerativeModel(
                model_name=MODEL_NAME,
                generation_config={"response_mime_type": "application/json", "temperature": 0.0}
            )
            
            try:
                print(f"      📡 Window {i+1}/{len(windows)}...")
                prompt = get_chunking_prompt(meta, window, context_memory)
                
                response = await asyncio.to_thread(model.generate_content, prompt)
                
                if response.usage_metadata:
                    t_in = response.usage_metadata.prompt_token_count
                    t_out = response.usage_metadata.candidates_token_count
                    total_in += t_in; total_out += t_out
                    print(f"      ✅ OK (Tok: {t_in}/{t_out})")
                
                new_items = json.loads(response.text)
                
                valid_content_chunks = []
                
                for item in new_items:
                    # 1. Memory Patching
                    if item.get('file_id') is not None: context_memory['file_id'] = item['file_id']
                    if item.get('book_page') is not None: context_memory['book_page'] = item['book_page']
                    if item.get('tg_msg_id') is not None: context_memory['tg_msg_id'] = item['tg_msg_id']
                    
                    if item.get('file_id') is None: item['file_id'] = context_memory['file_id']
                    if item.get('book_page') is None: item['book_page'] = context_memory['book_page']
                    if item.get('tg_msg_id') is None: item['tg_msg_id'] = context_memory['tg_msg_id']

                    # 2. Sort
                    ctype = item.get('chunk_type', 'content')
                    if ctype == 'bibliography':
                        book_data["bibliography"].append(item)
                    elif ctype == 'footnotes_definitions':
                        book_data["footnotes"].append(item)
                    else:
                        book_data["chunks"].append(item)
                        if item.get('relevance_score', 0) > 0.5:
                            valid_content_chunks.append(item)
                
                # 3. Update Last Chunk Context
                if valid_content_chunks:
                    context_memory['last_chunk_text'] = valid_content_chunks[-1]['chunk_content']
                    snippet = context_memory['last_chunk_text'][:50].replace('\n', ' ')
                    print(f"      🧠 Context updated: '{snippet}...'")
                
            except Exception as e:
                print(f"      ❌ Error: {e}")


        
        # --- ATTACH PAGE MEDIA (IMAGES) & OCR JSON TO CHUNKS ---
        page_media_map = book_data.get("page_media") or {}
        page_ocr_map = book_data.get("page_ocr") or {}
        page_scans_map = book_data.get("page_scans") or {}
        page_ocr_refs_map = book_data.get("page_ocr_refs") or {}

        def extract_tg_msg_id(link: str):
            """Извлекает числовой msg_id из строки вида 'tg://msg_id?id=1007'."""
            if not link:
                return None
            m = re.search(r"id=(\d+)", link)
            if not m:
                return None
            try:
                return int(m.group(1))
            except ValueError:
                return None

        for section_name in ("chunks", "bibliography", "footnotes"):
            for item in book_data.get(section_name, []):
                # Собираем все страницы, которые относятся к чанку
                page_keys = set()

                # Основная страница
                fid_primary = item.get("file_id")
                if fid_primary is not None:
                    try:
                        page_keys.add(int(fid_primary))
                    except (TypeError, ValueError):
                        pass

                # Дополнительная страница (если модель её вернула)
                fid_second = item.get("file_id_2")
                if fid_second is not None:
                    try:
                        page_keys.add(int(fid_second))
                    except (TypeError, ValueError):
                        pass

                # Fallback: старое поведение, если модель не заполнила file_id
                if not page_keys:
                    fid = item.get("file_id") or item.get("book_page")
                    try:
                        page_keys.add(int(fid))
                    except (TypeError, ValueError):
                        pass

                if not page_keys:
                    continue

                # 1) Связь чанка с исходными сканами
                scan_page_ids = sorted(page_keys)[:2]
                scan_tg_ids = []
                for pid in scan_page_ids:
                    tg_id = None
                    # Предпочитаем данные, уже лежащие в чанке
                    try:
                        if pid == int(item.get("file_id")) and item.get("tg_msg_id") not in (None, "N/A"):
                            tg_id = item.get("tg_msg_id")
                        elif pid == int(item.get("file_id_2")) and item.get("tg_msg_id_2") not in (None, "N/A"):
                            tg_id = item.get("tg_msg_id_2")
                    except Exception:
                        pass

                    if tg_id in (None, "N/A"):
                        tg_id = page_scans_map.get(pid)

                    if tg_id not in (None, "N/A"):
                        try:
                            tg_int = int(tg_id)
                            if tg_int not in scan_tg_ids:
                                scan_tg_ids.append(tg_int)
                        except ValueError:
                            continue

                if scan_page_ids:
                    item["scan_page_ids"] = scan_page_ids
                if scan_tg_ids:
                    item["scan_tg_msg_ids"] = scan_tg_ids

                # 2) Медиаконтент (иллюстрации и др. из APPX: Resources)
                internal_ids = []
                media_msg_ids = []
                for fid_int in sorted(page_keys):
                    page_media_list = page_media_map.get(fid_int) or []
                    if not page_media_list:
                        continue
                    for mobj in page_media_list:
                        mid = mobj.get("id")
                        if mid:
                            internal_ids.append(mid)
                        msg_id = extract_tg_msg_id(mobj.get("link", ""))
                        if msg_id is not None:
                            media_msg_ids.append(msg_id)

                if media_msg_ids:
                    item["page_media_ids"] = media_msg_ids
                    if not item.get("media_ids"):
                        item["media_ids"] = media_msg_ids

                existing_internal = item.get("media_internal_ids") or []
                if not isinstance(existing_internal, (list, tuple)):
                    existing_internal = [existing_internal]
                if internal_ids:
                    for mid in internal_ids:
                        if mid not in existing_internal:
                            existing_internal.append(mid)
                item["media_internal_ids"] = existing_internal

                # 3) OCR JSON для этих страниц
                ocr_ids = []
                for pid in scan_page_ids:
                    # 3a) Жёсткая привязка по заголовку (OCRjson в [File: ...])
                    header_ids = page_ocr_refs_map.get(pid) or []
                    for val in header_ids:
                        try:
                            vid = int(val)
                        except (TypeError, ValueError):
                            continue
                        if vid not in ocr_ids:
                            ocr_ids.append(vid)
                    # 3b) Фолбэк по APPX: Resources
                    page_ocr_list = page_ocr_map.get(pid) or []
                    for mobj in page_ocr_list:
                        msg_id = extract_tg_msg_id(mobj.get("link", ""))
                        if msg_id is not None and msg_id not in ocr_ids:
                            ocr_ids.append(msg_id)
                if ocr_ids:
                    item["ocr_tg_msg_ids"] = ocr_ids
# === SAVE BOOK TO SUPABASE ===
        try:
            print(f"💾 Saving book '{filename}' to Supabase...")
            insert_book_and_related(book_data)
        except Exception as e:
            print(f"[DB] ❌ DB Insert Error: {e}")


        # --- PREVIEW ---
        preview_md += f"# BOOK: {filename}\n"
        preview_md += "## 1. Metadata\n"
        for k, v in book_data["metadata"].items():
             preview_md += f"- **{k}**: {v}\n"
        
        preview_md += f"\n## 2. Content Chunks ({len(book_data['chunks'])})\n"
        for idx, ch in enumerate(book_data["chunks"]):
            score = ch.get('relevance_score', 0)
            icon = "🟢" if score > 0.7 else "🟡" if score > 0.4 else "🔴"
            if score < RELEVANCE_THRESHOLD: icon = "🗑️"
            
            flag = get_flag_emoji(ch.get('lang', ''))

            # Location string now also shows scan + OCR JSON bindings for debug
            scan_pages = ch.get('scan_page_ids') or []
            scan_tg = ch.get('scan_tg_msg_ids') or []
            ocr_ids = ch.get('ocr_tg_msg_ids') or []
            extra_parts = []
            if scan_pages:
                extra_parts.append(f"ScanPages: {scan_pages}")
            if scan_tg:
                extra_parts.append(f"ScanTG: {scan_tg}")
            if ocr_ids:
                extra_parts.append(f"OCRjson: {ocr_ids}")
            extra = ""
            if extra_parts:
                extra = " | " + " | ".join(extra_parts)

            loc_str = f"File: {ch.get('file_id')} | Book: {ch.get('book_page')} | Ref: {ch.get('tg_msg_id')}{extra}"
            
            size_icon = ""
            if len(ch.get('chunk_content', '')) > MAX_CHARS_PER_CHUNK: size_icon = "⚠️LONG"
            
            preview_md += f"### [{idx+1}] {icon} {flag} {loc_str} {size_icon}\n"
            preview_md += f"- **Topic**: {ch.get('semantic_header')}\n"
            preview_md += f"- **Tags**: {', '.join(ch.get('keywords', []))}\n"
            preview_md += f"- **Persons**: {', '.join(ch.get('persons', []))}\n"
            preview_md += f"- **Locations**: {', '.join(ch.get('locations', []))}\n"
            preview_md += f"- **Orthography**: {ch.get('orthography', 'modern')}\n"
            preview_md += f"- **Score**: {score} | **Len**: {len(ch.get('chunk_content', ''))}\n"

            media_ids = ch.get("media_ids") or []
            page_media_ids = ch.get("page_media_ids") or []

            def _as_str_list(values):
                out = []
                for v in values:
                    if v is None:
                        continue
                    out.append(str(v))
                return out

            if media_ids or page_media_ids:
                primary_str = ', '.join(_as_str_list(media_ids)) if media_ids else '—'
                page_str = ', '.join(_as_str_list(page_media_ids)) if page_media_ids else '—'
                preview_md += f"- **Media (primary)**: {primary_str}\n"
                preview_md += f"- **Media (page)**: {page_str}\n"
            preview_md += f"```text\n{ch.get('chunk_content')}\n```\n\n"

        if book_data["bibliography"]:
            preview_md += f"\n## 3. Bibliography ({len(book_data['bibliography'])})\n"
            for bib in book_data["bibliography"]:
                loc_str = f"[Ref: {bib.get('tg_msg_id')} | P: {bib.get('book_page')}]"
                preview_md += f"- {loc_str} {bib.get('chunk_content')[:300]}...\n"

        if book_data["footnotes"]:
            preview_md += f"\n## 4. Footnotes ({len(book_data['footnotes'])})\n"
            for fn in book_data["footnotes"]:
                 loc_str = f"[Ref: {fn.get('tg_msg_id')} | P: {fn.get('book_page')}]"
                 preview_md += f"- {loc_str} {fn.get('chunk_content')}\n"
        preview_md += "---\n\n"

    with open(output_preview_file, "w", encoding="utf-8") as f:
        f.write(preview_md)
    
    summary = (
        f"📊 **RAG Processing V60.1**\n"
        f"Sources: {len(files_to_process)}\n"
        f"Chunks: {len(book_data['chunks'])}\n"
        f"Tokens: {total_in} in / {total_out} out"
    )
    print(f"\n{summary}")
    
    try:
        await client.send_message(dest_ent, summary)
        await client.send_file(dest_ent, output_preview_file, caption="🧠 **RAG Data V60.1 (Fixed)**")
    except Exception as e:
        print(f"TG Upload Error: {e}")
        
    await client.disconnect()

# === RUN ===
await process_rag_pipeline()

[DB] 2025-12-03 20:25:32 Connecting to database for schema check...
[DB] 2025-12-03 20:25:33 Checking extensions (vector, pg_trgm, pgcrypto)...
[DB] 2025-12-03 20:25:33 Checking table books...
[DB] 2025-12-03 20:25:33 Creating table books...
[DB] 2025-12-03 20:25:34 Table books created.
[DB] 2025-12-03 20:25:34 Checking table media...
[DB] 2025-12-03 20:25:34 Creating table media...
[DB] 2025-12-03 20:25:34 Table media created.
[DB] 2025-12-03 20:25:34 Checking table chunks...
[DB] 2025-12-03 20:25:34 Creating table chunks...
[DB] 2025-12-03 20:25:34 Table chunks created.
[DB] 2025-12-03 20:25:34 Checking/Updating match_chunks function...
[DB] 2025-12-03 20:25:34 Schema check complete.
📂 Found 1. Processing: 1.

📘 Processing: Попов_М_,_Попадин_А_От_Канта_до_Твердого_соединения_далей,_к_ландшафту.md
   ✂️ Windows: 3 (Overlap: 1000)
      📡 Window 1/3...
      ✅ OK (Tok: 4459/7613)
      🧠 Context updated: 'Положение оси моста окончательно не установлено, о...'
      📡 Window 2/3...
    